In [1]:
# matpltlib 库以及参数
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'axes.labelsize': 'large'})
matplotlib.rcParams.update({'xtick.labelsize': 'large'})
matplotlib.rcParams.update({'ytick.labelsize': 'large'})
matplotlib.rcParams.update({'legend.fontsize': 'large'})
matplotlib.rcParams.update({'errorbar.capsize': 3})

#其他的库
import numpy as np
import pandas as pd
import scipy as sp 

#封装进度条的函数
from tqdm import tqdm    
#导入自己的函数
import mydef

In [2]:
df=pd.read_csv('./DATA/matched_data_step2.csv')
df

,SDSS_NAME,RA,DEC,REDSHIFT,g_SDSS,r_SDSS,i_SDSS,LOGBH,LOGEDD_RATIO,LOGLBOL,ra,dec,g_HSC,r_HSC,i_HSC,hsc_id
0,222230.16+010231.4,335.625693,1.042081,0.2235,19.480,18.707,18.221,8.562371,-1.670831,44.991911,335.625695,1.042063,19.239025,18.737987,18.374371,42644584253119735
1,134826.86-005943.9,207.111950,-0.995528,0.2340,19.600,18.677,18.105,8.468839,-1.452045,45.117165,207.111954,-0.995535,20.204891,19.244598,18.704962,41192941141645641
2,134459.44-001559.5,206.247690,-0.266537,0.2449,17.669,17.585,17.438,7.804190,-0.662985,45.241575,206.247703,-0.266531,18.520733,18.283943,18.000725,41188010519184869
3,225757.22-002608.3,344.488429,-0.435656,0.2509,18.951,18.656,18.411,8.305248,-1.562993,44.842626,344.488417,-0.435649,18.982300,18.745213,18.441834,41602247229986243
4,135908.00+002731.8,209.783356,0.458855,0.2570,18.858,18.440,18.222,8.119494,-1.139233,45.080632,209.783372,0.458862,18.799702,18.566313,18.287029,42270595680863627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5153,144407.63-010152.7,221.031804,-1.031321,4.5180,22.504,20.194,19.296,8.903947,0.447643,47.451960,221.031806,-1.031322,22.090027,19.956085,19.308536,41231969509481510
5154,233610.47-005814.8,354.043660,-0.970795,4.5211,22.930,20.830,19.945,9.851813,-1.093724,46.858459,354.043685,-0.970779,22.535259,20.530994,19.759323,41628072868342015
5155,141914.18-015012.6,214.809118,-1.836846,4.5863,23.063,20.140,19.073,10.409191,-1.177417,47.332144,214.809096,-1.836840,22.975964,19.938438,19.280464,40145531762140284
5156,222050.80+001959.1,335.211698,0.333103,4.7160,23.962,21.722,20.119,9.646668,-0.637838,47.109201,335.211688,0.333105,24.093718,21.735556,20.580292,42639344393019967


# 按照红移分bin

In [3]:
df.sort_values(by='REDSHIFT',ascending=True)
df['red_bin_index']=pd.cut(df.index,bins=np.arange(0,5214+100,100),right=False,labels=range(53))#RIGHT=FALSE表示左闭右开
df['org_index']=df.index
df_r=df.set_index(['red_bin_index','org_index'])

In [4]:
df_r

SDSS_NAME          RA       DEC  REDSHIFT  \
red_bin_index org_index                                                       
0             0          222230.16+010231.4  335.625693  1.042081    0.2235   
              1          134826.86-005943.9  207.111950 -0.995528    0.2340   
              2          134459.44-001559.5  206.247690 -0.266537    0.2449   
              3          225757.22-002608.3  344.488429 -0.435656    0.2509   
              4          135908.00+002731.8  209.783356  0.458855    0.2570   
...                                     ...         ...       ...       ...   
51            5153       144407.63-010152.7  221.031804 -1.031321    4.5180   
              5154       233610.47-005814.8  354.043660 -0.970795    4.5211   
              5155       141914.18-015012.6  214.809118 -1.836846    4.5863   
              5156       222050.80+001959.1  335.211698  0.333103    4.7160   
              5157       000749.16+004119.6    1.954853  0.688786    4.8327   

                         g_SDSS  r_SDSS  i_SDSS      LOGBH  LOGEDD_RATIO  \
red_bin_index org_index                                                    
0             0          19.480  18.707  18.221   8.562371     -1.670831   
              1          19.600  18.677  18.105   8.468839     -1.452045   
              2          17.669  17.585  17.438   7.804190     -0.662985   
              3          18.951  18.656  18.411   8.305248     -1.562993   
              4          18.858  18.440  18.222   8.119494     -1.139233   
...                         ...     ...     ...        ...           ...   
51            5153       22.504  20.194  19.296   8.903947      0.447643   
              5154       22.930  20.830  19.945   9.851813     -1.093724   
              5155       23.063  20.140  19.073  10.409191     -1.177417   
              5156       23.962  21.722  20.119   9.646668     -0.637838   
              5157       24.895  21.502  20.060  10.046684     -1.104105   

                           LOGLBOL          ra       dec      g_HSC  \
red_bin_index org_index                                               
0             0          44.991911  335.625695  1.042063  19.239025   
              1          45.117165  207.111954 -0.995535  20.204891   
              2          45.241575  206.247703 -0.266531  18.520733   
              3          44.842626  344.488417 -0.435649  18.982300   
              4          45.080632  209.783372  0.458862  18.799702   
...                            ...         ...       ...        ...   
51            5153       47.451960  221.031806 -1.031322  22.090027   
              5154       46.858459  354.043685 -0.970779  22.535259   
              5155       47.332144  214.809096 -1.836840  22.975964   
              5156       47.109201  335.211688  0.333105  24.093718   
              5157       47.042949    1.954850  0.688767  25.168055   

                             r_HSC      i_HSC             hsc_id  
red_bin_index org_index                                           
0             0          18.737987  18.374371  42644584253119735  
              1          19.244598  18.704962  41192941141645641  
              2          18.283943  18.000725  41188010519184869  
              3          18.745213  18.441834  41602247229986243  
              4          18.566313  18.287029  42270595680863627  
...                            ...        ...                ...  
51            5153       19.956085  19.308536  41231969509481510  
              5154       20.530994  19.759323  41628072868342015  
              5155       19.938438  19.280464  40145531762140284  
              5156       21.735556  20.580292  42639344393019967  
              5157       21.931934  20.393879  41649655079001595  

[5158 rows x 16 columns]

# 向其中扩充3+3列方便计算


In [5]:
df_r['g_SDSS-HSC']=df_r['g_SDSS']-df_r['g_HSC']
df_r['r_SDSS-HSC']=df_r['r_SDSS']-df_r['r_HSC']
df_r['i_SDSS-HSC']=df_r['i_SDSS']-df_r['i_HSC']

df_r['MBH']=10**df_r['LOGBH']
df_r['EDD_RATIO']=10**df_r['LOGEDD_RATIO']
df_r['LBOL']=10**df_r['LOGLBOL']

df_r['half_g_SDSS+HSC']=(df_r['g_SDSS']+df_r['g_HSC'])/2
df_r

SDSS_NAME          RA       DEC  REDSHIFT  \
red_bin_index org_index                                                       
0             0          222230.16+010231.4  335.625693  1.042081    0.2235   
              1          134826.86-005943.9  207.111950 -0.995528    0.2340   
              2          134459.44-001559.5  206.247690 -0.266537    0.2449   
              3          225757.22-002608.3  344.488429 -0.435656    0.2509   
              4          135908.00+002731.8  209.783356  0.458855    0.2570   
...                                     ...         ...       ...       ...   
51            5153       144407.63-010152.7  221.031804 -1.031321    4.5180   
              5154       233610.47-005814.8  354.043660 -0.970795    4.5211   
              5155       141914.18-015012.6  214.809118 -1.836846    4.5863   
              5156       222050.80+001959.1  335.211698  0.333103    4.7160   
              5157       000749.16+004119.6    1.954853  0.688786    4.8327   

                         g_SDSS  r_SDSS  i_SDSS      LOGBH  LOGEDD_RATIO  \
red_bin_index org_index                                                    
0             0          19.480  18.707  18.221   8.562371     -1.670831   
              1          19.600  18.677  18.105   8.468839     -1.452045   
              2          17.669  17.585  17.438   7.804190     -0.662985   
              3          18.951  18.656  18.411   8.305248     -1.562993   
              4          18.858  18.440  18.222   8.119494     -1.139233   
...                         ...     ...     ...        ...           ...   
51            5153       22.504  20.194  19.296   8.903947      0.447643   
              5154       22.930  20.830  19.945   9.851813     -1.093724   
              5155       23.063  20.140  19.073  10.409191     -1.177417   
              5156       23.962  21.722  20.119   9.646668     -0.637838   
              5157       24.895  21.502  20.060  10.046684     -1.104105   

                           LOGLBOL  ...      r_HSC      i_HSC  \
red_bin_index org_index             ...                         
0             0          44.991911  ...  18.737987  18.374371   
              1          45.117165  ...  19.244598  18.704962   
              2          45.241575  ...  18.283943  18.000725   
              3          44.842626  ...  18.745213  18.441834   
              4          45.080632  ...  18.566313  18.287029   
...                            ...  ...        ...        ...   
51            5153       47.451960  ...  19.956085  19.308536   
              5154       46.858459  ...  20.530994  19.759323   
              5155       47.332144  ...  19.938438  19.280464   
              5156       47.109201  ...  21.735556  20.580292   
              5157       47.042949  ...  21.931934  20.393879   

                                    hsc_id  g_SDSS-HSC  r_SDSS-HSC  \
red_bin_index org_index                                              
0             0          42644584253119735    0.240975   -0.030987   
              1          41192941141645641   -0.604891   -0.567598   
              2          41188010519184869   -0.851733   -0.698943   
              3          41602247229986243   -0.031300   -0.089213   
              4          42270595680863627    0.058298   -0.126313   
...                                    ...         ...         ...   
51            5153       41231969509481510    0.413973    0.237915   
              5154       41628072868342015    0.394741    0.299006   
              5155       40145531762140284    0.087036    0.201562   
              5156       42639344393019967   -0.131718   -0.013556   
              5157       41649655079001595   -0.273055   -0.429934   

                         i_SDSS-HSC           MBH  EDD_RATIO          LBOL  \
red_bin_index org_index                                                      
0             0           -0.153371  3.650659e+08   0.021339  9.815460e+44   
              1           

In [6]:
df_r.to_csv('./DATA/redshift_bin_org.csv')

# 删掉一些不必要的参数

In [7]:
col=['SDSS_NAME','RA','DEC','hsc_id','ra','dec']
df_r=df_r.drop(col,axis=1)

In [8]:
df_r

REDSHIFT  g_SDSS  r_SDSS  i_SDSS      LOGBH  \
red_bin_index org_index                                                
0             0            0.2235  19.480  18.707  18.221   8.562371   
              1            0.2340  19.600  18.677  18.105   8.468839   
              2            0.2449  17.669  17.585  17.438   7.804190   
              3            0.2509  18.951  18.656  18.411   8.305248   
              4            0.2570  18.858  18.440  18.222   8.119494   
...                           ...     ...     ...     ...        ...   
51            5153         4.5180  22.504  20.194  19.296   8.903947   
              5154         4.5211  22.930  20.830  19.945   9.851813   
              5155         4.5863  23.063  20.140  19.073  10.409191   
              5156         4.7160  23.962  21.722  20.119   9.646668   
              5157         4.8327  24.895  21.502  20.060  10.046684   

                         LOGEDD_RATIO    LOGLBOL      g_HSC      r_HSC  \
red_bin_index org_index                                                  
0             0             -1.670831  44.991911  19.239025  18.737987   
              1             -1.452045  45.117165  20.204891  19.244598   
              2             -0.662985  45.241575  18.520733  18.283943   
              3             -1.562993  44.842626  18.982300  18.745213   
              4             -1.139233  45.080632  18.799702  18.566313   
...                               ...        ...        ...        ...   
51            5153           0.447643  47.451960  22.090027  19.956085   
              5154          -1.093724  46.858459  22.535259  20.530994   
              5155          -1.177417  47.332144  22.975964  19.938438   
              5156          -0.637838  47.109201  24.093718  21.735556   
              5157          -1.104105  47.042949  25.168055  21.931934   

                             i_HSC  g_SDSS-HSC  r_SDSS-HSC  i_SDSS-HSC  \
red_bin_index org_index                                                  
0             0          18.374371    0.240975   -0.030987   -0.153371   
              1          18.704962   -0.604891   -0.567598   -0.599962   
              2          18.000725   -0.851733   -0.698943   -0.562725   
              3          18.441834   -0.031300   -0.089213   -0.030833   
              4          18.287029    0.058298   -0.126313   -0.065029   
...                            ...         ...         ...         ...   
51            5153       19.308536    0.413973    0.237915   -0.012536   
              5154       19.759323    0.394741    0.299006    0.185677   
              5155       19.280464    0.087036    0.201562   -0.207464   
              5156       20.580292   -0.131718   -0.013556   -0.461292   
              5157       20.393879   -0.273055   -0.429934   -0.333879   

                                  MBH  EDD_RATIO          LBOL  \
red_bin_index org_index                                          
0             0          3.650659e+08   0.021339  9.815460e+44   
              1          2.943333e+08   0.035315  1.309680e+45   
              2          6.370741e+07   0.217278  1.744116e+45   
              3          2.019519e+08   0.027353  6.960267e+44   
              4          1.316722e+08   0.072572  1.204015e+45   
...                               ...        ...           ...   
51            5153       8.015801e+08   2.803127  2.831133e+47   
              5154       7.109073e+09   0.080589  7.218703e+46   
              5155       2.565611e+10   0.066463  2.148544e+47   
              5156       4.432702e+09   0.230230  1.285883e+47   
              5157       1.113484e+10   0.078685  1.103950e+47   

                         half_g_SDSS+HSC  
red_bin_index org_index                   
0             0                19.359513  
              1                19.902446  
              2                18.094866  
              3                18.966650  
              4                18.828851  
..

In [9]:
df_r.to_csv('./DATA/redshift_bin.csv')

In [10]:
df_s=pd.read_csv('./DATA/redshift_bin.csv',index_col=[0,1])#读取的时候一定要读取两个头哦